# *Tilt angle* e *Theta map*

Este código utiliza a biblioteca de modelagem e inversão de dados geofísicos denominada [Fatiando a Terra](http://www.fatiando.org/index.html) para calcular o *tilt angle* (Miller e Singh, 1994; Verduzco et al., 2004; Salem et al., 2008; Cooper, 2014) e o *theta map* (Wijns, et al., 2005) da anomalia de campo total produzida por um prisma com seção horizontal poligonal.

* Miller, H. G. e Singh, V. 1994. Potential field tilt—a new concept for location of potential field sources. Journal of Applied Geophysics, v. 32, Issues 2–3, p. 213-217. doi: [10.1016/0926-9851(94)90022-1](http://www.sciencedirect.com/science/article/pii/0926985194900221)

* Verduzco, B., Fairhead, J. D., Green, C. M. e MacKenzie, C. 2004. New insights into magnetic derivatives for structural mapping. The Leading Edge, 23(2), 116-119. doi: [10.1190/1.1651454](http://dx.doi.org/10.1190/1.1651454)

* Wijns, C., Perez, C. Kowalczyk, P. 2005. Theta map: Edge detection in magnetic data. GEOPHYSICS, 70(4), L39-L43. doi: [10.1190/1.1988184](http://dx.doi.org/10.1190/1.1988184)

* Salem, A., Williams, S., Fairhead,, D., Smith, R. e Ravat, D. 2008. Interpretation of magnetic data using tilt-angle derivatives. GEOPHYSICS, 73(1), L1-L10. doi: [10.1190/1.2799992](http://dx.doi.org/10.1190/1.2799992)

* Cooper, G. R. J. 2014. The automatic determination of the location, depth, and dip of contacts from aeromagnetic data. GEOPHYSICS, 79(3), J35-J41. doi: [10.1190/geo2013-0181.1](http://dx.doi.org/10.1190/geo2013-0181.1)

### Coisas para importar

In [1]:
#%matplotlib inline
import numpy as np
from fatiando import mesher, gridder, utils
from fatiando.gravmag import polyprism
from fatiando.vis import mpl

### Pontos onde são calculados os dados

In [2]:
shape = (100, 100) # número de pontos ao longo de x e y
area = [-5000, 5000, -5000, 5000] # x mínimo, x máximo, y mínimo e y máximo da área
xp, yp, zp = gridder.regular(area, shape, z=-100) # grade regular localizada em z = -100 m

xc = 0.5*(area[1] + area[0])
yc = 0.5*(area[3] + area[2])

### Inclinação e a declinação do campo geomagnético local

In [3]:
inc = -20. # graus
dec = -20. # graus
amp = 23500.0 # nT

In [4]:
F = utils.ang2vec(amp, inc, dec) # vetor campo geomagnético local

In [5]:
print F

[ 20751.02220665  -7552.75441382  -8037.47336815]


### Modelo

In [6]:
#mpl.close('all')
#axis = mpl.figure().gca()
#mpl.axis('scaled')
#vertices = mpl.draw_polygon(area, axis, xy2ne=True)

In [7]:
#print vertices

In [8]:
vertices = [[ 2301.25523013, -3085.77405858],
 [ 2693.51464435, -2248.9539749],
 [ 2798.11715481, -261.50627615],
 [ 2222.80334728,  1359.83263598],
 [ 1412.13389121,  2013.59832636],
 [  183.05439331,  2667.36401674],
 [ -941.42259414,  2850.41841004],
 [-2562.76150628,  2850.41841004],
 [-3478.0334728,   1961.29707113],
 [-3974.89539749,   444.56066946],
 [-4053.34728033, -1072.17573222],
 [-3347.28033473, -2771.9665272 ],
 [-2745.81589958, -2588.91213389],
 [-2510.46025105,  -862.9707113 ],
 [-2588.91213389 ,  366.10878661],
 [ -967.57322176,  1176.77824268],
 [  -26.15062762 , 1098.32635983],
 [ 1150.62761506  ,  78.45188285],
 [ 1098.32635983, -1595.18828452],
 [ 1046.0251046 , -2588.91213389],
 [  575.31380753, -3451.88284519],
 [  339.958159 ,  -3922.59414226],
 [ 1490.58577406, -4471.75732218],
 [ 2353.55648536, -4497.90794979],
 [ 2955.0209205,  -4341.0041841 ],
 [ 3922.59414226, -4001.0460251 ],
 [ 4367.15481172, -3347.28033473],
 [ 3948.74476987, -2798.11715481],
 [ 2955.0209205 , -2850.41841004]]

In [9]:
incs = inc # inclinacao da magnetizacao do corpo (em graus)
decs = 0. # declinacao da magnetizacao do corpo (em graus)
amps = 10. # intensidade da magnetizacao do corpo (em A/m)

In [10]:
model = [mesher.PolygonalPrism(vertices, 200, 1200, {'magnetization': utils.ang2vec(amps, incs, decs)})]

### Anomalia de campo total (em nT)

In [11]:
act = polyprism.tf(xp, yp, zp, model, inc, dec)

### Derivadas espaciais da anomalia de campo total (em nT/m)

In [12]:
delta = 10.

dx_act = (polyprism.tf(xp + delta, yp, zp, model, inc, dec) - \
          polyprism.tf(xp - delta, yp, zp, model, inc, dec))/(2.*delta)

dy_act = (polyprism.tf(xp, yp + delta, zp, model, inc, dec) - \
          polyprism.tf(xp, yp - delta, zp, model, inc, dec))/(2.*delta)

dz_act = (polyprism.tf(xp, yp, zp + delta, model, inc, dec) - \
          polyprism.tf(xp, yp, zp - delta, model, inc, dec))/(2.*delta)

### Amplitude do gradiente horizontal

In [13]:
grad_xy = np.sqrt(dx_act**2 + dy_act**2)

### Amplitude do gradiente

In [14]:
grad = np.sqrt(dx_act**2 + dy_act**2 + dz_act**2)

### Tilt angle

In [15]:
tilt= np.arctan2(dz_act,grad_xy)

### Theta map

In [16]:
theta = np.arccos(grad_xy/grad)

### Plot das derivadas ao longo dos eixos x, y e z

In [17]:
mpl.close('all')
mpl.figure(figsize=(14,15))

mpl.subplot(3,2,1)
mpl.title("Tilt angle (radianos)")
mpl.axis('scaled')
mpl.contourf(yp, xp, tilt, shape, 15, cmap = mpl.get_cmap('RdBu_r'), 
            vmin = -0.5*np.pi, vmax = 0.5*np.pi)
mpl.colorbar()
#mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,2)
mpl.title("Derivada em x da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dx_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,3)
mpl.title("Derivada em y da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dy_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,4)
mpl.title("Derivada em z da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dz_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,5)
mpl.title("Amplitude do gradiente horizontal (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, grad_xy, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,2,6)
mpl.title("Amplitude do gradiente (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, grad, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.tight_layout()

mpl.savefig('prismas_mag_derivadas_tilt.png', dpi=300)

#mpl.show()

<img src='prismas_mag_derivadas_tilt.png', width=1200>

In [19]:
mpl.close('all')
mpl.figure(figsize=(7,15))

mpl.subplot(3,1,1)
mpl.title("Tilt angle (radianos)")
mpl.axis('scaled')
mpl.contourf(yp, xp, tilt, shape, 15, cmap = mpl.get_cmap('RdBu_r'), 
            vmin = -0.5*np.pi, vmax = 0.5*np.pi)
mpl.colorbar()
#mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,1,2)
mpl.title("Amplitude do gradiente (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, grad, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(3,1,3)
mpl.title("Theta map (radianos)")
mpl.axis('scaled')
mpl.contourf(yp, xp, theta, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
#mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.tight_layout()

mpl.savefig('prismas_mag_derivadas_tilt_theta.png', dpi=300)

<img src='prismas_mag_derivadas_tilt_theta.png', width=600>